In [1]:
from mmdet.apis import init_detector, inference_detector, show_result_pyplot, single_gpu_test
import mmcv
from mmcv import Config

import pandas as pd
import os 
import numpy as np
import matplotlib.pyplot as plt

In [2]:
cfg = Config.fromfile('../configs/detectoRS_cascade_rcnn-hand-bone.py')

cfg.model.roi_head.bbox_head[0].num_classes = 13
cfg.model.roi_head.bbox_head[1].num_classes = 13
cfg.model.roi_head.bbox_head[2].num_classes = 13

checkpoint_file = 'hand_bone_exps/latest.pth'
cfg.gpu_ids = [1]

In [1]:
model = init_detector(cfg, checkpoint_file, device='cuda:2')
model.CLASSES = ['radius', 'ulna', 'metacarpal1', 'metacarpal3', 'metacarpal5', 
               'nearly-phalanx1','nearly-phalanx3', 'nearly-phalanx5', 'middle-phalanx3', 'middle-phalanx5', 
              'far-phalanx1', 'far-phalanx3', 'far-phalanx5']

In [143]:
import os 
import random
import matplotlib.pyplot as plt

In [145]:
def infer_coor(test_file):
    test_img = mmcv.imread(test_file)
    test_img_ = mmcv.imresize(test_img, (int(test_img.shape[0]), int(test_img.shape[1])))
    result = inference_detector(model, test_img_)

    return result

In [148]:
def parse_result(filename ,res, df, threshold=0.5):
    assert len(res) == 13
    columns=['filename', 'bone_type', 'min_x', 'min_y', 'max_x', 'max_y', 'conf']
    for i,_class in enumerate(model.CLASSES):
        if len(res[i]) <= 0:
            continue
        for boxs in res[i]:
            tt = dict()
            tt['filename'] = filename
            tt['bone_type'] = _class
            for j in range(5):
                tt[columns[2+j]] = boxs[j]
            if tt['conf'] >= threshold:
                df = df.append(tt, ignore_index=True)
    return df

In [150]:
def infer(test_file, save_dir):
    test_img = mmcv.imread(test_file)
    test_img_ = mmcv.imresize(test_img, (int(test_img.shape[0]), int(test_img.shape[1])))
    save_file = os.path.join(save_dir, os.path.basename(test_file))

    result = inference_detector(model, test_img_)

    img = model.show_result(test_img_, result, score_thr=0.4,show=False, bbox_color='red', 
                            text_color='red', thickness=1, font_scale=0.3)
    idx = int(os.path.basename(test_file).split('.')[0])
    mmcv.imwrite(img, save_file)
    print('\r\r', save_file, end='')

In [153]:
test_dir = '/fdr/fdr/datasets/hand_bone/resized_last_use'
test_list = os.listdir(test_dir)

In [154]:
df = pd.DataFrame(columns=['filename', 'bone_type', 'min_x', 'min_y', 'max_x', 'max_y', 'conf'])

for file in test_list:
    val_file = os.path.join(test_dir, file)
    res = infer_coor(val_file)
    df = parse_result(file.split('.')[0], res, df)
    print('\r\r', val_file, end='')

 /fdr/fdr/datasets/hand_bone/resized_last_use/013710.jpg

In [155]:
df.to_csv('result/hand_bone_last_use_coor_boxes.csv', index=False)

In [156]:
saved_dir = 'result/hand_bone_resized_last_use_result'

for file in test_list:
    val_file = os.path.join(test_dir, file)
    infer(val_file, saved_dir)

 result/hand_bone_resized_last_use_result/013710.jpg result/hand_bone_resized_last_use_result/015780.jpg